# Transfer Learning in CNN

# Step-1:

In [1]:
import pandas as pd
import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

# Step-2:

In [2]:
now = datetime.datetime.now

batch_size = 128
num_classes = 5
epochs = 5

img_rows, img_cols = 28,28
filters = 32
pool_size = 2
kernel_size = 3

# Step-3:

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gt5 = x_train[y_train >= 5]
y_train_gt5 = y_train[y_train >= 5]-5
x_test_gt5 = x_test[y_test >= 5]
y_test_gt5 = y_test[y_test >= 5]-5

In [5]:
x_train_lt5.shape

(30596, 28, 28)

In [6]:
x_train_gt5.shape

(29404, 28, 28)

# Step-4:

In [7]:
feature_layers = [
    Conv2D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same', input_shape=(img_rows, img_cols, 1)),
    Conv2D(filters=filters, kernel_size=kernel_size, activation='relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten()                                                             
]

# Step-5:

In [8]:
classification_layers = [
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')                                                             
]

# Step-6:

In [9]:
model = Sequential(feature_layers + classification_layers)

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 13, 13, 32)        0         
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 128)               692352    
                                                        

# Step-7:

In [11]:
def train_model(model, train, test, num_classes):

    train[0] = train[0].reshape(train[0].shape[0], img_rows, img_cols, 1).astype('float32') / 255
    test[0] = test[0].reshape(test[0].shape[0], img_rows, img_cols, 1).astype('float32') / 255

    print(train[0].shape)
    print(test[0].shape)

    s = pd.Series(train[1])
    train[1] = pd.get_dummies(s)
    train[1] = train[1].values

    s = pd.Series(test[1])
    test[1] = pd.get_dummies(s)
    test[1] = test[1].values

    model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])
    train_start_time = now()
    model.fit(train[0], train[1], epochs=epochs, batch_size=batch_size, verbose=2)

    print('\n')
    print('--------------------------------------------------------------------------------')
    print('\n')

    print('Training time: %s' % (now() - train_start_time))

    score = model.evaluate(test[0], test[1], verbose=0)

    print('Test loss =', score[0])
    print('Test accuracy =', score[1]*100)

# Step-8:

In [12]:
train_model(model, [x_train_gt5, y_train_gt5], [x_test_gt5, y_test_gt5], num_classes)

(29404, 28, 28, 1)
(4861, 28, 28, 1)
Epoch 1/5
230/230 - 15s - loss: 1.6072 - accuracy: 0.2205 - 15s/epoch - 67ms/step
Epoch 2/5
230/230 - 15s - loss: 1.5902 - accuracy: 0.2822 - 15s/epoch - 67ms/step
Epoch 3/5
230/230 - 15s - loss: 1.5697 - accuracy: 0.3564 - 15s/epoch - 64ms/step
Epoch 4/5
230/230 - 15s - loss: 1.5493 - accuracy: 0.4221 - 15s/epoch - 64ms/step
Epoch 5/5
230/230 - 15s - loss: 1.5219 - accuracy: 0.4864 - 15s/epoch - 64ms/step


--------------------------------------------------------------------------------


Training time: 0:01:15.014763
Test loss = 1.493734359741211
Test accuracy = 77.28862166404724


# Step-9:

In [13]:
for layers in feature_layers:
    layers.trainable = False

# Step-10:

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 26, 26, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 13, 13, 32)        0         
                                                                 
 flatten (Flatten)           (None, 5408)              0         
                                                                 
 dense (Dense)               (None, 128)               692352    
                                                        

# Step-11:

In [15]:
train_model(model, [x_train_lt5, y_train_lt5], [x_test_lt5, y_test_lt5], num_classes)

(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
240/240 - 4s - loss: 1.5795 - accuracy: 0.2919 - 4s/epoch - 16ms/step
Epoch 2/5
240/240 - 4s - loss: 1.5451 - accuracy: 0.3821 - 4s/epoch - 15ms/step
Epoch 3/5
240/240 - 4s - loss: 1.5087 - accuracy: 0.4656 - 4s/epoch - 15ms/step
Epoch 4/5
240/240 - 4s - loss: 1.4731 - accuracy: 0.5346 - 4s/epoch - 15ms/step
Epoch 5/5
240/240 - 4s - loss: 1.4378 - accuracy: 0.5912 - 4s/epoch - 15ms/step


--------------------------------------------------------------------------------


Training time: 0:00:18.337648
Test loss = 1.4002854824066162
Test accuracy = 82.19497799873352


# Step-12:

In [16]:
feature_layers_reverse = [
    Conv2D(filters=filters, kernel_size=kernel_size, activation='relu', padding='same', input_shape=(img_rows, img_cols, 1)),
    Conv2D(filters=filters, kernel_size=kernel_size, activation='relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten()                                                             
]

In [17]:
classification_layers_reverse = [
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')                                                             
]

In [18]:
model_reverse = Sequential(feature_layers_reverse + classification_layers_reverse)
model_reverse.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 conv2d_3 (Conv2D)           (None, 26, 26, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 13, 13, 32)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 13, 13, 32)        0         
                                                                 
 flatten_1 (Flatten)         (None, 5408)              0         
                                                                 
 dense_2 (Dense)             (None, 128)               692352    
                                                      

In [ ]:
train_model(model_reverse, [x_train_lt5, y_train_lt5], [x_test_lt5, y_test_lt5], num_classes)

(30596, 28, 28, 1)
(5139, 28, 28, 1)
Epoch 1/5
240/240 - 16s - loss: 1.5973 - accuracy: 0.2872 - 16s/epoch - 68ms/step
Epoch 2/5
240/240 - 16s - loss: 1.5579 - accuracy: 0.3699 - 16s/epoch - 65ms/step
Epoch 3/5
240/240 - 16s - loss: 1.5157 - accuracy: 0.4693 - 16s/epoch - 67ms/step
Epoch 4/5


In [ ]:
for layers in feature_layers_reverse:
    layers.trainable = False

In [ ]:
model_reverse.summary()

In [ ]:
train_model(model_reverse, [x_train_gt5, y_train_gt5], [x_test_gt5, y_test_gt5], num_classes)